### Implementing FunkSVD

In this notebook we will take a look at writing our own function that performs FunkSVD, which will follow the steps you saw in the previous video.  If you find that you aren't ready to tackle this task on your own, feel free to skip to the following video where you can watch as I walk through the steps.

To test our algorithm, we will run it on the subset of the data you worked with earlier.  Run the cell below to get started.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import sparse
import svd_tests as t
%matplotlib inline

# Read in the datasets
movies = pd.read_csv('data/movies_clean.csv')
reviews = pd.read_csv('data/reviews_clean.csv')

del movies['Unnamed: 0']
del reviews['Unnamed: 0']

# Create user-by-item matrix
user_items = reviews[['user_id', 'movie_id', 'rating', 'timestamp']]
user_by_movie = user_items.groupby(['user_id', 'movie_id'])['rating'].max().unstack()

# Create data subset
user_movie_subset = user_by_movie[[73486, 75314,  68646, 99685]].dropna(axis=0)
ratings_mat = np.matrix(user_movie_subset)
print(ratings_mat)

[[ 10.  10.  10.  10.]
 [ 10.   4.   9.  10.]
 [  8.   9.  10.   5.]
 [  9.   8.  10.  10.]
 [ 10.   5.   9.   9.]
 [  6.   4.  10.   6.]
 [  9.   8.  10.   9.]
 [ 10.   5.   9.   8.]
 [  7.   8.  10.   8.]
 [  9.   5.   9.   7.]
 [  9.   8.  10.   8.]
 [  9.  10.  10.   9.]
 [ 10.   9.  10.   8.]
 [  5.   8.   5.   8.]
 [ 10.   8.  10.  10.]
 [  9.   9.  10.  10.]
 [  9.   8.   8.   8.]
 [ 10.   8.   1.  10.]
 [  5.   6.  10.  10.]
 [  8.   7.  10.   7.]]


`1.` You will use the **user_movie_subset** matrix to show that your FunkSVD algorithm will converge.  In the below cell, use the comments and document string to assist you as you complete writing your own function to complete FunkSVD.  You may also want to try to complete the function on your own without the assistance of comments.  You may feel free to remove and add to the function in any way that gets you a working solution! 

**Notice:** There isn't a sigma matrix in this version of the matrix factorization.

In [3]:
def FunkSVD(ratings_mat, latent_features=4, learning_rate=0.0001, iters=100):
    '''
    This function performs matrix factorization using a basic form of FunkSVD with no regularization
    
    INPUT:
    ratings_mat - (numpy array) a matrix with users as rows, movies as columns, and ratings as values
    latent_features - (int) the number of latent features used
    learning_rate - (float) the learning rate 
    iters - (int) the number of iterations
    
    OUTPUT:
    user_mat - (numpy array) a user by latent feature matrix
    movie_mat - (numpy array) a latent feature by movie matrix
    '''
    
    # Set up useful values to be used through the rest of the function
    n_users = ratings_mat.shape[0] # number of rows in the matrix
    n_movies = ratings_mat.shape[1] # number of movies in the matrix
    num_ratings = np.count_nonzero(~np.isnan(ratings_mat)) # total number of ratings in the matrix
    
    # initialize the user and movie matrices with random values
    # helpful link: https://docs.scipy.org/doc/numpy/reference/generated/numpy.random.rand.html
    user_mat = np.random.rand(n_users, latent_features) # user matrix filled with random values of shape user x latent 
    movie_mat = np.random.rand(latent_features, n_movies) # movie matrix filled with random values of shape latent x movies
    
    # initialize sse at 0 for first iteration
    sse_accum = 0
    
    # header for running results
    print("Optimization Statistics")
    print("Iterations | Mean Squared Error ")
    
    # for each iteration
    for iteration in range(iters):
        # update our sse
        old_sse = sse_accum
        sse_accum = 0
        
        # For each user-movie pair
        for i in range(n_users):
            for j in range(n_movies):      
                # if the rating exists
                if ratings_mat[i, j] > 0:
                    # compute the error as the actual minus the dot product of the user and movie latent features
                    diff = ratings_mat[i, j] - np.dot(user_mat[i, :], movie_mat[:, j])
                    # Keep track of the total sum of squared errors for the matrix
                    sse_accum += diff**2
                    # update the values in each matrix in the direction of the gradient
                    for k in range(latent_features):
                        user_mat[i, k] += learning_rate * (2*diff*movie_mat[k, j])
                        movie_mat[k, j] += learning_rate * (2*diff*user_mat[i, k])
        # print results for iteration
        print("%d \t\t %f" % (iteration+1, sse_accum / num_ratings))
    return user_mat, movie_mat 

`2.` Try out your function on the **user_movie_subset** dataset.  First try 4 latent features, a learning rate of 0.005, and 10 iterations.  When you take the dot product of the resulting U and V matrices, how does the resulting **user_movie** matrix compare to the original subset of the data?

In [4]:
# use your function with 4 latent features, lr of 0.005 and 10 iterations
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=4, learning_rate=0.005, iters=10) 

Optimization Statistics
Iterations | Mean Squared Error 
1 		 38.282868
2 		 11.819820
3 		 3.630385
4 		 2.722593
5 		 2.601704
6 		 2.547171
7 		 2.498791
8 		 2.446443
9 		 2.387536
10 		 2.321070


In [5]:
#Compare the predicted and actual results
print(np.dot(user_mat, movie_mat))
print(ratings_mat)

[[ 10.11170472   8.34806514  10.81034846  10.25567588]
 [  8.61507066   7.38812339   9.01320682   8.86879397]
 [  7.47020206   7.49618276   8.23709309   7.68833093]
 [  9.54643988   8.35475901  10.2873795    9.32226659]
 [  8.4223346    7.18630413   9.07587522   8.65215192]
 [  6.64436902   5.69629249   7.6943101    6.74431792]
 [  9.3143541    7.79448038   9.61390749   9.34456547]
 [  8.5169383    7.19449812   8.18227977   8.38079888]
 [  8.28726082   7.47820496   9.0326264    8.46140431]
 [  7.70018469   6.42832254   8.29485812   7.63906972]
 [  9.18986758   7.76121489   9.19341917   8.8070597 ]
 [  9.22433911   8.79217603  10.04460351   9.58292332]
 [  9.46925986   8.44156095   9.62822501   9.05106186]
 [  6.44757365   6.11455896   6.84985545   6.61721779]
 [  9.67456163   8.44992568  10.22975907   9.81982145]
 [  9.40855274   8.42756057  10.55340497   9.70324737]
 [  8.68163091   7.62424771   8.24013965   8.25978171]
 [  7.80005249   6.36156579   6.57822549   7.32659956]
 [  8.1900

**The predicted ratings from the dot product are already starting to look a lot like the original data values even after only 10 iterations.  You can see some extreme low values that are not captured well yet.  The 5 in the second to last row in the first column is predicted as an 8, and the 4 in the second row and second column is predicted to be a 7.  Clearly the model is not done learning, but things are looking good.**

`3.` Let's try out the function again on the **user_movie_subset** dataset.  This time we will again use 4 latent features and a learning rate of 0.005.  However, let's bump up the number of iterations to 250.  When you take the dot product of the resulting U and V matrices, how does the resulting **user_movie** matrix compare to the original subset of the data?  What do you notice about your error at the end of the 250 iterations?

In [6]:
#use your function with 4 latent features, lr of 0.005 and 250 iterations
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=4, learning_rate=0.005, iters=250)

Optimization Statistics
Iterations | Mean Squared Error 
1 		 48.364687
2 		 18.879264
3 		 4.843104
4 		 2.900391
5 		 2.755027
6 		 2.716905
7 		 2.686156
8 		 2.654219
9 		 2.619159
10 		 2.579982
11 		 2.535820
12 		 2.485839
13 		 2.429273
14 		 2.365491
15 		 2.294072
16 		 2.214900
17 		 2.128238
18 		 2.034802
19 		 1.935783
20 		 1.832815
21 		 1.727867
22 		 1.623072
23 		 1.520502
24 		 1.421946
25 		 1.328732
26 		 1.241632
27 		 1.160870
28 		 1.086211
29 		 1.017109
30 		 0.952860
31 		 0.892742
32 		 0.836102
33 		 0.782416
34 		 0.731297
35 		 0.682484
36 		 0.635815
37 		 0.591199
38 		 0.548583
39 		 0.507937
40 		 0.469233
41 		 0.432442
42 		 0.397526
43 		 0.364449
44 		 0.333169
45 		 0.303649
46 		 0.275856
47 		 0.249763
48 		 0.225347
49 		 0.202589
50 		 0.181467
51 		 0.161959
52 		 0.144033
53 		 0.127652
54 		 0.112763
55 		 0.099308
56 		 0.087215
57 		 0.076404
58 		 0.066787
59 		 0.058274
60 		 0.050769
61 		 0.044180
62 		 0.038414
63 		 0.033383
64 		

In [7]:
#Compare the predicted and actual results
print(np.dot(user_mat, movie_mat))
print(ratings_mat)

[[ 10.00000421  10.00000211  10.00000136   9.99999545]
 [ 10.00000473   4.00000239   9.00000151   9.99999481]
 [  8.0000039    9.00000189  10.00000117   4.99999574]
 [  9.00000126   8.00000053  10.00000023   9.99999854]
 [  9.99999804   4.9999988    8.99999917   9.0000021 ]
 [  5.99999946   3.99999976   9.99999977   6.00000043]
 [  9.00000358   8.00000146  10.00000092   8.99999626]
 [  9.9999987    4.99999933   8.99999947   8.00000124]
 [  6.99999753   7.99999863   9.99999903   8.00000261]
 [  8.99999976   4.99999975   8.99999982   7.00000025]
 [  8.99999347   7.99999657   9.99999776   8.00000705]
 [  8.99999934   9.99999961   9.99999975   9.00000074]
 [  9.9999969    8.99999863   9.99999908   8.00000318]
 [  4.99999895   7.99999941   4.99999964   8.00000121]
 [ 10.00000042   8.00000018  10.00000018   9.99999962]
 [  8.99999931   8.99999972   9.99999984  10.00000073]
 [  9.00000226   8.00000121   8.0000008    7.99999753]
 [  9.99999912   7.99999959   0.99999971  10.00000091]
 [  4.9999

**In this case, we were able to completely reconstruct the item-movie matrix to obtain an essentially 0 mean squared error. I obtained 0 MSE on iteration 165.**

The last time we placed an **np.nan** value into this matrix the entire svd algorithm in python broke.  Let's see if that is still the case using your FunkSVD function.  In the below cell, I have placed a nan into the first cell of your numpy array.  

`4.` Use 4 latent features, a learning rate of 0.005, and 250 iterations.  Are you able to run your SVD without it breaking (something that was not true about the python built in)?  Do you get a prediction for the nan value?  What is your prediction for the missing value? Use the cells below to answer these questions.

In [8]:
# Here we are placing a nan into our original subset matrix
ratings_mat[0, 0] = np.nan
ratings_mat

matrix([[ nan,  10.,  10.,  10.],
        [ 10.,   4.,   9.,  10.],
        [  8.,   9.,  10.,   5.],
        [  9.,   8.,  10.,  10.],
        [ 10.,   5.,   9.,   9.],
        [  6.,   4.,  10.,   6.],
        [  9.,   8.,  10.,   9.],
        [ 10.,   5.,   9.,   8.],
        [  7.,   8.,  10.,   8.],
        [  9.,   5.,   9.,   7.],
        [  9.,   8.,  10.,   8.],
        [  9.,  10.,  10.,   9.],
        [ 10.,   9.,  10.,   8.],
        [  5.,   8.,   5.,   8.],
        [ 10.,   8.,  10.,  10.],
        [  9.,   9.,  10.,  10.],
        [  9.,   8.,   8.,   8.],
        [ 10.,   8.,   1.,  10.],
        [  5.,   6.,  10.,  10.],
        [  8.,   7.,  10.,   7.]])

In [9]:
# run SVD on the matrix with the missing value
#use your function with 4 latent features, lr of 0.005 and 250 iterations
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=4, learning_rate=0.005, iters=250) 

Optimization Statistics
Iterations | Mean Squared Error 
1 		 47.069893
2 		 18.815081
3 		 5.242756
4 		 2.970598
5 		 2.720206
6 		 2.657167
7 		 2.615751
8 		 2.576473
9 		 2.535275
10 		 2.490818
11 		 2.442419
12 		 2.389615
13 		 2.332062
14 		 2.269517
15 		 2.201833
16 		 2.128958
17 		 2.050937
18 		 1.967923
19 		 1.880184
20 		 1.788129
21 		 1.692338
22 		 1.593597
23 		 1.492929
24 		 1.391607
25 		 1.291132
26 		 1.193167
27 		 1.099421
28 		 1.011506
29 		 0.930782
30 		 0.858227
31 		 0.794361
32 		 0.739238
33 		 0.692495
34 		 0.653452
35 		 0.621229
36 		 0.594860
37 		 0.573381
38 		 0.555894
39 		 0.541606
40 		 0.529838
41 		 0.520024
42 		 0.511703
43 		 0.504501
44 		 0.498114
45 		 0.492301
46 		 0.486861
47 		 0.481631
48 		 0.476473
49 		 0.471272
50 		 0.465925
51 		 0.460344
52 		 0.454447
53 		 0.448161
54 		 0.441420
55 		 0.434160
56 		 0.426326
57 		 0.417866
58 		 0.408736
59 		 0.398896
60 		 0.388319
61 		 0.376984
62 		 0.364883
63 		 0.352023
64 		

In [10]:
# Run this cell to see if you were able to predict for the missing value
preds = np.dot(user_mat, movie_mat)
print("The predicted value for the missing rating is {}:".format(preds[0,0]))
print()
print("The actual value for the missing rating is {}:".format(ratings_mat[0,0]))
print()
assert np.isnan(preds[0,0]) == False
print("That's right! You just predicted a rating for a user-movie pair that was never rated!")
print("But if you look in the original matrix, this was actually a value of 10. Not bad!")

The predicted value for the missing rating is 9.117731666676143:

The actual value for the missing rating is nan:

That's right! You just predicted a rating for a user-movie pair that was never rated!
But if you look in the original matrix, this was actually a value of 10. Not bad!


Now let's extend this to a more realistic example. Unfortunately, running this function on your entire user-movie matrix is still not something you likely want to do on your local machine.  However, we can see how well this example extends to 1000 users.  In the above portion, you were using a very small subset of data with no missing values.

`5.` Given the size of this matrix, this will take quite a bit of time.  Consider the following hyperparameters: 4 latent features, 0.005 learning rate, and 20 iterations.  Grab a snack, take a walk, and this should be done running in a bit.

In [12]:
# Setting up a matrix of the first 1000 users with movie ratings
first_1000_users = np.matrix(user_by_movie.head(1000))

# perform funkSVD on the matrix of the top 1000 users
#fit to 1000 users with 4 latent features, lr of 0.005, and 20 iterations
user_mat, movie_mat = FunkSVD(first_1000_users, latent_features=4, learning_rate=0.005, iters=20) 

Optimization Statistics
Iterations | Mean Squared Error 
1 		 23.542644
2 		 10.801413
3 		 7.422145
4 		 5.731299
5 		 4.662723
6 		 3.913547
7 		 3.354514
8 		 2.920178
9 		 2.573644
10 		 2.292102
11 		 2.060311
12 		 1.867486
13 		 1.705704
14 		 1.568986
15 		 1.452727
16 		 1.353309
17 		 1.267844
18 		 1.193991
19 		 1.129827
20 		 1.073762


`6.` Now that you have a set of predictions for each user-movie pair,  let's answer a few questions about your results. Provide the correct values for each of the variables below, and check your solutions using the tests below.

In [14]:
# Replace each of the comments below with the correct values
# How many actual ratings exist in first_1000_users
num_ratings = np.count_nonzero(~np.isnan(first_1000_users))
print("The number of actual ratings in the first_1000_users is {}.".format(num_ratings))
print()

# How many ratings did we make for user-movie pairs that didn't actually have ratings
ratings_for_missing = first_1000_users.shape[0]*first_1000_users.shape[1] - num_ratings 
print("The number of ratings made for user-movie pairs that didn't have ratings is {}".format(ratings_for_missing))

The number of actual ratings in the first_1000_users is 10852.

The number of ratings made for user-movie pairs that didn't have ratings is 31234148


In [15]:
# Test your results against the solution
assert num_ratings == 10852, "Oops!  The number of actual ratings doesn't quite look right."
assert ratings_for_missing == 31234148, "Oops!  The number of movie-user pairs that you made ratings for that didn't actually have ratings doesn't look right."

# Make sure you made predictions on all the missing user-movie pairs
preds = np.dot(user_mat, movie_mat)
assert np.isnan(preds).sum() == 0
print("Nice job!  Looks like you have predictions made for all the missing user-movie pairs! But I still have one question... How good are they?")

Nice job!  Looks like you have predictions made for all the missing user-movie pairs! But I still have one question... How good are they?
